In [2]:
import sys
import pandas as pd
import numpy as np
import scanpy as sc
import pathlib as pt
import matplotlib as mpl
import matplotlib.pyplot as plt

scr_path = '/nfs/team292/hm11/endo_GLND/starsolo/GLND/Thyroid/snippets'
sys.path.append(scr_path)
import singleCellFun as sf

f,_ = sf.makeOutoutFolders()
sf.nbInit(f)

# output path

In [4]:
output_path = pt.Path().absolute().parent / 'cpdh_input_T21_vs_2n_thy_imm'
print(output_path)

/nfs/team292/hm11/endo_GLND/starsolo/GLND/Thyroid/matching_DS_Oct24/all_soupX/T21_vs_2n/cpdh_T21_vs_2n/cpdh_input_T21_vs_2n_thy_imm


# read celltype edgeR DGE

read up regulated genes,
concatenate lists, 
make celltype - gene list

In [5]:
# THRESH 
FDR_THRESH = 0.05
LOGFC_THRESH = 0.2
MIN_GENES_THRESH = 20
MIN_PER_THRESH = 10

FDRstr = str(FDR_THRESH).replace('.','')
LOGFCstr = str(LOGFC_THRESH).replace('.','')
MIN_GENESstr = str(MIN_GENES_THRESH)
MIN_PERstr = str(MIN_PER_THRESH)

In [7]:
# load DEG
fname = '../../data/combined_DEG_results_forHassan.csv'
df = pd.read_csv(fname)
print(f'{df.shape} significant genes')

dge = df[(df['logFC'] > LOGFC_THRESH) & (df['FDR'] < FDR_THRESH)]

# genes expressed in late age-group in both cell types
dge = dge[dge['comparison'].isin(['fTFC1_14_20','fTFC2_14_20','analysis2_fTFC1','analysis2_fTFC2'])]
# dge = dge[dge['comparison'].isin(['fTFC1_11_13'])]
dge['cell_type'] = dge['comparison']

update = {'analysis2_fTFC1':'thy_TH_processing', 'analysis2_fTFC2' : 'thy_Lumen_forming',
          'fTFC1_14_20':'thy_TH_processing', 'fTFC2_14_20' : 'thy_Lumen_forming'}
dge = dge.replace(update)
# reorder columns
dge = dge.rename(columns={'geneSym':'gene'})
col = dge.columns
cols2front = ['cell_type','gene']
dge = dge[cols2front + list(set(col)-set(cols2front))]
# concat celltypes - karyotypes
dge['cell_type'] = dge['cell_type'] + str('.') + str('T21')
# print
print(dge.shape)
print(dge['cell_type'].unique())
display(dge.head(3))

(365, 17) significant genes
(294, 18)
['thy_TH_processing.T21' 'thy_Lumen_forming.T21']


,cell_type,gene,PValue,Unnamed: 0,isTF,logFC,F,ensID,comparison,isCosmic,chr,isCSM,cosmicTier,tumourType,direction,isTSG,FDR,logCPM
0,thy_TH_processing.T21,PDXK,9.560567e-09,ENSG00000160209,False,0.904324,143.840007,ENSG00000160209,thy_TH_processing,False,chr21,False,NaN,NaN,T21_up,False,0.000063,5.666743
1,thy_TH_processing.T21,AGPAT3,1.009336e-08,ENSG00000160216,False,0.955210,142.627732,ENSG00000160216,thy_TH_processing,False,chr21,False,NaN,NaN,T21_up,False,0.000063,6.036753
2,thy_TH_processing.T21,USP16,3.848646e-08,ENSG00000156256,False,0.635044,115.505041,ENSG00000156256,thy_TH_processing,False,chr21,False,NaN,NaN,T21_up,False,0.000161,6.881284


In [8]:
fname = f'dge_mergeMi_T21_vs_2n_{MIN_GENESstr}min_{MIN_PER_THRESH}per_p1Luz.tsv'
print(f'output filename: {fname}')
dge.to_csv(output_path / fname, index=False, sep='\t')

output filename: dge_mergeMi_T21_vs_2n_20min_10per_p1Luz.tsv


# read microenvironment
read adata, subset for interesting celltypes, export adata, export metadata, export microenv

In [11]:
# run cellphone for all celltypes except of cycling cells
adata = sc.read('../../../workingObj/cdata_all.h5ad')
sf.summaryTable(adata,['karyotype','pcw','donor','sex','cells'])

/home/jovyan/my-conda-envs/sc_Harm/lib/python3.10/site-packages/anndata/__init__.py:42: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(


[INFO] - total cells: 213141


/nfs/team292/hm11/endo_GLND/starsolo/GLND/Thyroid/snippets/singleCellFun.py:107: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  try: tempAnn['pcw'] = tempAnn['pcw'].values.astype(int)


,karyotype,sex,donor,pcw,cells
0,2n,male,Hrv188,11,11727
1,2n,male,Hrv195,11,7492
2,T21,male,Hrv256,11,3916
3,T21,male,Hrv257,11,8743
4,2n,female,Hrv197,12,9892
5,2n,male,Hrv116,12,4839
6,2n,male,Hrv218,12,8981
7,T21,female,Hrv265,12,7862
8,T21,male,Hrv266,12,9242
9,2n,female,Hrv194,14,10882


In [61]:
immune_in

[('imm_NK_cells', '14-20'),
 ('imm_NK_cells', '11-13'),
 ('imm_Cycling_T', '14-20'),
 ('imm_T_cells', '14-20'),
 ('imm_Monocytes', '14-20'),
 ('imm_Macrophages', '14-20'),
 ('imm_Cycling_T', '11-13'),
 ('imm_DC2', '14-20'),
 ('imm_T_cells', '11-13'),
 ('imm_Mast_cells', '14-20'),
 ('imm_Monocytes', '11-13'),
 ('imm_Cycling_NK_cells', '14-20'),
 ('imm_Cycling_NK_cells', '11-13'),
 ('imm_ILC', '14-20'),
 ('imm_ILC', '11-13'),
 ('imm_Mast_cells', '11-13'),
 ('imm_B_cells', '14-20'),
 ('imm_DC2', '11-13')]

In [81]:
# get immune cell type with more that MIN_CELLS in T21 
MIN_CELLS = 20
counts_df = adata.obs.loc[(adata.obs['lineage'] == 'Immune') & 
                         (adata.obs['karyotype'] == 'T21'), ['celltype','age_group']].reset_index(drop=True).value_counts().reset_index()
immune_in = counts_df.loc[counts_df['count'] > MIN_CELLS, 'celltype'].unique().tolist()
print(f'len of filtered immune cluster {len(immune_in)}')
immune_in


len of filtered immune cluster 10


['imm_NK_cells',
 'imm_Cycling_T',
 'imm_T_cells',
 'imm_Monocytes',
 'imm_Macrophages',
 'imm_DC2',
 'imm_Mast_cells',
 'imm_Cycling_NK_cells',
 'imm_ILC',
 'imm_B_cells']

In [82]:
# zoom in to thy|eno|mes
cluster_in = ['thy_TH_processing','thy_Lumen_forming'] + immune_in
cluster_in = [s for s in cluster_in if 'cycling' not in s.lower()] #remove cycling cells
print(f'len cluster in {len(cluster_in)}')
cluster_in

len cluster in 10


['thy_TH_processing',
 'thy_Lumen_forming',
 'imm_NK_cells',
 'imm_T_cells',
 'imm_Monocytes',
 'imm_Macrophages',
 'imm_DC2',
 'imm_Mast_cells',
 'imm_ILC',
 'imm_B_cells']

In [85]:
# update fields (replace '-' by '_')
adata.obs['celltype'] = adata.obs['celltype'].replace({'thy_Lumen-forming' : 'thy_Lumen_forming'})
# subset adata
adata = adata[adata.obs['celltype'].isin(cluster_in)]
adata.shape

/tmp/ipykernel_1468/1090616985.py:2: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  adata.obs['celltype'] = adata.obs['celltype'].replace({'thy_Lumen-forming' : 'thy_Lumen_forming'})


(52767, 30997)

# write adata

In [86]:
adata.write(output_path / 'thy_imm_cdata.h5ad')

# write metadata

In [87]:
tmp = adata.obs['celltype'].astype(str) + '.' + adata.obs['karyotype'].astype(str)
tmp.reset_index().to_csv(output_path / 'cdata_all_MATCHING_metadata_p1Luz.tsv', index=False, sep='\t')

# write microenv

In [88]:
pd.concat([tmp, adata.obs['karyotype']], axis=1).drop_duplicates().reset_index(drop=True).to_csv(output_path / 'cdata_all_MATCHING_microenvironment_p1Luz.tsv',index=False, sep='\t')